<a href="https://colab.research.google.com/github/Michele-Maestrini/FusionCore/blob/main/notebooks/04_Forensic_Stress_Test1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# FusionCore v0 — Prognostic Architecture & Benchmarking

**Status:** `Architecture Definition`

**Author:** Michele Maestrini

**Previous Context:** [Feature Engineering]

## (1) Executive Objective: Forensic Audit & Structural Validation

The primary objective of this notebook is to conduct a **Forensic Stress Test** of the predictive framework to ensure that Remaining Useful Life (RUL) estimations are derived from physical sensor signals rather than metadata artefacts. Following the discovery of near-perfect predictive accuracy in the initial baseline, this notebook serves as a **Methodological Gate** to identify, isolate, and remediate **Structural Temporal Leakage**.



### 1.1 The Forensic Litmus Test
To ensure the model is fit for safety-critical aerospace deployment, we move beyond standard accuracy metrics and subject the architecture to three validation pillars:

* **Temporal Blindness Audit:** Systematically removing timestamp and index metadata to determine if the model is "cheating" by interpolating a linear countdown rather than interpreting thermodynamic degradation.
* **Physics-Aware Feature Engineering:** Augmenting the dataset with kinematic derivatives—**Delta ($\Delta X$)** and **Rolling Means ($\bar{X}$)**—to force the model to focus on the *velocity* of degradation (Condition-Based Monitoring) rather than simple asset age.
* **Anomaly Sensitivity Stress Test:** Injecting synthetic sensor failures into "healthy" early-life assets to verify if the model prioritises physical evidence over the elapsed "clock". A model that ignores a catastrophic sensor spike in favour of the asset's age is deemed a safety failure.

### 1.2 Success Criteria & Domain Metrics
* **Honest Baseline Establishment:** Transitioning from an artificial $10^{-9}$ MAE to a physically grounded **MAE (~2.28)** that accurately reflects sensor noise and stochastic wear.
* **Prognostic Logic Verification:** Success is defined by the model's ability to reduce RUL predictions in direct response to physical sensor spikes, demonstrating a "Prognostic Reasoning" capability.
* **Data Readiness (v1_physics):** The final output is a sanitised, high-fidelity `.parquet` dataset, stripped of all temporal shortcuts and ready for a rigorous comparison against State-of-the-Art (SOTA) architectures in the subsequent phase.

### Dependency Installation

In [ ]:
# AutoGluon for the ensemble baseline
# PyTorch Forecasting for the TFT implementation
!pip install -q autogluon.timeseries pytorch-forecasting

### Libraries

In [ ]:
import numpy as np
import pandas as pd
import torch

from google.colab import drive
import os
import pickle

from autogluon.timeseries import TimeSeriesDataFrame, TimeSeriesPredictor
from autogluon.tabular import TabularPredictor

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics import r2_score, mean_absolute_error

## (2) Environment Setup & Forensic Audit Framework

To execute a rigorous stress test of our prognostic baseline, we must establish a stable computation environment capable of handling high-dimensional telemetry and synthetic anomaly injection. This section ensures the "Digital Thread" between the raw C-MAPSS data and our forensic validation remains unbroken.

### 2.1 Dependency Management
We utilise a specialised suite of libraries designed for statistical validation and automated benchmarking:
* **AutoGluon (Forensic Baseline):** Used specifically as a "black-box" diagnostic tool to identify structural leakage and establish an initial performance ceiling.
* **Scikit-Learn:** For implementing the shuffled tabular regression and calculating the "Honest" MAE baseline.
* **Matplotlib & Seaborn:** To generate the residual plots and prognostic visualisations required to identify statistical heteroscedasticity.

### 2.2 Data Recovery & Path Synchronisation
The processed sensor data and engineered kinematics are retrieved from the Project Path (`/content/drive/MyDrive/PI/Datasets`). This section synchronises our local environment with the Google Drive repository, ensuring that all subsequent audits are performed on the same regime-normalised telemetry validated in previous steps.

### 2.3 Structural Integrity Audit
Before initiating the stress tests, we perform a validation of the data schema. This ensures:
1. **Feature Consistency:** All engineered rolling statistics and deltas are correctly mapped across the four C-MAPSS subsets.
2. **Target Alignment:** RUL labels are correctly associated with their respective sensor snapshots, ensuring that our "Honest" MAE calculation is mathematically sound.
3. **Leakage Prevention:** Verification that metadata indices (item_id, timestamp) are available for grouping but are excluded from the training tensors to prevent temporal cheating.

In [ ]:
drive.mount('/content/drive')

In [ ]:
# Updating to match your specific Project Path
DATA_PATH = "/content/drive/MyDrive/PI/Datasets"
TENSOR_FILE = 'FusionCore_Final_Tensors.pkl'

def load_final_tensors(path):
    """
    Retrieves the 3D tensors and target vectors from the established project path.
    """
    target_path = os.path.join(path, TENSOR_FILE)
    if os.path.exists(target_path):
        with open(target_path, 'rb') as f:
            data = pickle.load(f)
        print(f"✅ Tensors successfully recovered from: {target_path}")
        return data
    else:
        # Diagnostic check: list available files if the path fails
        print(f"❌ Target not found. Contents of {path}:")
        try:
            print(os.listdir(path))
        except Exception as e:
            print(f"Directory inaccessible: {e}")
        raise FileNotFoundError(f"Could not locate {TENSOR_FILE}")

# Execute recovery
final_tensors = load_final_tensors(DATA_PATH)

In [ ]:
# Modern PyTorch API (2.0+) for precision control
# 'high' enables TF32 on Ampere GPUs for a performance boost with minimal precision loss.
# 'highest' uses full float32 (IEEE) for maximum numerical rigour.
torch.set_float32_matmul_precision('high')

# Silencing the CuDNN TF32 warning specifically
torch.backends.cudnn.allow_tf32 = True

In [ ]:
# --- 2.1 TENSOR SCHEMA AUDIT ---

def audit_tensor_shapes(tensors):
    """
    Summarises the shapes of 3D input tensors (X) and target vectors (y).
    """
    audit_results = []

    for ds_name in tensors.keys():
        for split in tensors[ds_name].keys():
            X_shape = tensors[ds_name][split]['X'].shape
            y_shape = tensors[ds_name][split]['y'].shape

            audit_results.append({
                'Dataset': ds_name,
                'Split': split,
                'Samples (N)': X_shape[0],
                'Window (W)': X_shape[1],
                'Features (F)': X_shape[2],
                'Target Shape': y_shape
            })

    # Create a tabulated view
    audit_df = pd.DataFrame(audit_results)
    display(audit_df)

# Execute the audit
audit_tensor_shapes(final_tensors)

In [ ]:
# --- 2.2 FULL TENSOR SCHEMA AUDIT ---

def audit_all_subsets(tensors):
    """
    Summarises the dimensions for all CMAPSS sub-datasets.
    """
    audit_data = []
    for ds in sorted(tensors.keys()):
        for split in ['train', 'test']: # Focusing on primary splits
            if split in tensors[ds]:
                X_shape = tensors[ds][split]['X'].shape
                y_shape = tensors[ds][split]['y'].shape
                audit_data.append({
                    'Dataset': ds,
                    'Split': split,
                    'Samples (N)': X_shape[0],
                    'Window (W)': X_shape[1],
                    'Features (F)': X_shape[2],
                    'Target': y_shape
                })

    audit_df = pd.DataFrame(audit_data)
    print("📋 Final Handshake: Tensor Dimensions Across All Subsets")
    return audit_df

# Display the audit
audit_all_subsets(final_tensors)

## (3) AutoGluon Baseline: Establishing the Predictive Ceiling - Stress Test

To provide a rigorous benchmark for our custom deep learning architectures, we utilise **AutoGluon-TimeSeries** as our primary baseline. AutoGluon employs an automated stacking and ensembling strategy—blending multiple state-of-the-art models such as DeepAR, PatchTST, and various statistical estimators—to determine the maximum attainable accuracy for the C-MAPSS dataset.

### 3.1 The "Prognostic Stress Test" (FD002 & FD004)
While we will evaluate the entire suite (FD001–FD004), we utilise **FD002** and **FD004** as our primary "Stress Tests" for architectural validation. These subsets are uniquely challenging due to:
* **Operating Complexity:** They incorporate six distinct flight regimes. A successful model must distinguish between sensor fluctuations caused by mechanical degradation and those caused by shifts in altitude, Mach number, or throttle settings.
* **Fault Diversity:** Unlike the simpler FD001/003 subsets, these include two distinct failure modes, requiring the model to generalise across different physical breakdown mechanisms (e.g., High-Pressure Compressor vs. Fan degradation).

### 3.2 Data Re-Engineering: The Long-Format Requirement
AutoGluon-TimeSeries operates on a **Long-Format** `TimeSeriesDataFrame` rather than the 3D tensors ($N \times W \times F$) required by LSTMs and Transformers. To ensure a fair comparison, we must perform a precise data re-engineering step:
1. **Inverse Transformation:** We will reconstruct the tabular temporal sequence from our windowed tensors, ensuring that the temporal order of the 30-cycle windows is preserved.
2. **Schema Alignment:** We will map the cycle indices and Unit IDs back to a 2D structure, where each row represents a discrete cycle in an engine's life.
3. **Target Parity:** We must ensure the **Piecewise Linear RUL** (capped at 125 cycles) is correctly associated with the final cycle of each sequence to maintain mathematical consistency across all models in our triumvirate.

By establishing this automated baseline first, we define the "Machine Benchmark"—the score that our custom-tuned **TFT** and **TSMixer** architectures must aim to outperform or, at the very least, match with significantly lower computational overhead.

In [ ]:
# --- 3.1 TENSOR TO LONG-FORMAT CONVERSION ---

def transform_to_autogluon_format(tensors, dataset_name='FD002', split='train'):
    """
    Converts 3D tensors back to a Long-Format DataFrame for AutoGluon.
    We extract the final cycle of each window to map to the RUL target.
    """
    X = tensors[dataset_name][split]['X']
    y = tensors[dataset_name][split]['y']

    # In Notebook 03, we established the feature names.
    # For the audit, we'll use generic indices or map back if names are stored.
    num_features = X.shape[2]
    feature_cols = [f"feat_{i}" for i in range(num_features)]

    # We take the last cycle of each 30-step window
    # This represents the 'Current State' that corresponds to the RUL label
    X_last_cycle = X[:, -1, :]

    df = pd.DataFrame(X_last_cycle, columns=feature_cols)
    df['target_rul'] = y

    # IMPORTANT: AutoGluon needs a 'timestamp' and 'item_id'.
    # Since we lost 'unit_id' in the raw tensor, we need to ensure the order
    # is preserved or re-map from the original split_data if available.
    # For the baseline, we treat the sequence as a single continuous id
    # or map back using the sample indices.

    df['item_id'] = 1 # Temporary ID for the baseline run
    df['timestamp'] = np.arange(len(df)) # Sequential cycles

    return TimeSeriesDataFrame.from_data_frame(
        df,
        id_column="item_id",
        timestamp_column="timestamp"
    )

# Prepare the data for the AutoGluon Stress Test
train_data_ag = transform_to_autogluon_format(final_tensors, 'FD002', 'train')
print(f"✅ AutoGluon DataFrame Created. Shape: {train_data_ag.shape}")

In [ ]:
# --- 3.2 INITIATING THE BASELINE STRESS TEST ---

# Define the prediction horizon (for RUL, we typically focus on the next step)
# However, AutoGluon requires a prediction_length >= 1
PREDICTION_LENGTH = 1

predictor = TimeSeriesPredictor(
    target="target_rul",
    prediction_length=PREDICTION_LENGTH,
    eval_metric="RMSE",
    path="autogluon_models_fd002",
    verbosity=2
)

# Train using the high_quality preset to establish the benchmark
# We set a time limit to manage Google Colab resources effectively
predictor.fit(
    train_data_ag,
    presets="high_quality",
    time_limit=1800,  # 30-minute stress test limit
)

In [ ]:
# --- 3.3 EVALUATING THE LEADERBOARD ---
leaderboard = predictor.leaderboard(train_data_ag)
display(leaderboard)

### Model Performance Analysis: Predictive Maintenance Leaderboard

## 1. The Performance Gap: Tabular vs. Deep Learning
The **DirectTabular** and **WeightedEnsemble** models are achieving near-zero error (order of $10^{-9}$), while heavyweights like **Temporal Fusion Transformer (TFT)** and **Chronos** are trailing by orders of magnitude.

* **The Critique:** In aerospace predictive maintenance, a "perfect" score often signals **data leakage**. If your features include variables that are mathematically derived from the target or if the test set contains "future" information leaked through windowing/shuffling, your model isn't learning physics—it's learning an identity function.
* **The Gap:** Why is TFT at $-0.13$ while DirectTabular is at $-2.47 \times 10^{-9}$? TFT is designed to handle complex non-linearities and global patterns. If a simple tabular model beats it this decisively, your underlying signal might be highly linear or the dataset size might be too small for the TFT to converge on the manifold.

## 2. Chronos and Pre-trained Models
It is notable that **Chronos2 (Large)** outperformed the **FineTuned Small** version.

* **The Methodology Flaw:** The `ChronosSmallFineTuned` model has a significantly worse test score ($-1.499$) compared to its validation score ($-0.687$). This is a textbook case of **overfitting**. In a reliability context, this is dangerous; it suggests the model has "memorised" the training degradation curves and fails to generalise to the unique sensor noise or initial conditions of the test units.
* **Inference Latency:** Note the `pred_time_test`. Chronos2 is roughly **38× slower** than `RecursiveTabular`. For real-time health management in an edge environment (like an orbital satellite or a turbine controller), that latency penalty is a deal-breaker unless the accuracy gain is massive—which here, it isn't.

## 3. The "Naïve" Baselines
**SeasonalNaive** and **AutoETS** are sitting at scores of approximately $-1.0$.

* **The Interpretation:** Since these are baselines, they define the "floor" of acceptable performance. Any model scoring worse than $-1.0$ (like your fine-tuned Chronos or the Regressor variant) is effectively performing worse than a model that just guesses the last known seasonal value.

### Comparative Analysis of Top Models
| Model | Test Score (MSE/MAE) | Fit Time (s) | Risk Factor |
| :--- | :--- | :--- | :--- |
| **WeightedEnsemble** | $\approx 0$ | 0.17 | High risk of leakage/Overfit |
| **DirectTabular** | $\approx 0$ | 2.46 | Strongest contender for production |
| **TFT** | $-0.138$ | 85.20 | Computationally expensive; needs tuning |
| **Chronos2** | $-0.257$ | 1.11 | High inference latency |

### Questions for the Methodology
To push this into a "production-ready" aerospace context, you should address the following:
* **Stationarity:** Did you check for unit roots or non-stationarity in the sensor data? Tabular models often struggle with raw trends compared to state-space models.
* **The Zero-Error Phenomenon:** Can you justify a score of $10^{-9}$? In aerospace, sensor noise usually dictates a higher error floor. Verify that you haven't included the "Time-to-Failure" (TTF) as a feature.
* **Prognostics Horizon:** Is the model predicting just the next timestamp (one-step-ahead) or the entire RUL curve? One-step prediction is often trivial for maintenance planning.

### Investigating the "Near-Perfect" Score

To identify why `DirectTabular` is achieving $10^{-9}$, we must scrutinise two areas: **Windowing Logic** and **Feature Leakage**.

#### 1. The Windowing Logic
In time-series for predictive maintenance, a common error is "shuffling before windowing." If you create windows and then perform a random train-test split, the model might see windows that overlap significantly in time.



* **The Test:** Are you using a **Time-Series Split** (Walk-forward validation) where the test set is strictly chronologically after the training set?
* **The Risk:** If you used a random split on overlapping windows, the model is **interpolating** between data points it has already seen in a slightly different window.

#### 2. Feature Importance & Leakage
The fact that `DirectTabular` is winning suggests the relationship is highly structural.

* **The "ID" Problem:** If the engine/unit ID or the "Cycle Number" is treated as a raw numerical feature, the model might be mapping the target directly to the cycle count rather than learning degradation physics.
* **Lag Leakage:** If your features include $Y_{t-1}$ and you are predicting $Y_t$ in a system with very high sampling frequency, the "score" will look perfect because the signal hasn't changed enough to challenge the model.

#### 3. Definitive Answers on Methodology
To move from suspicion to rigour, we must perform these three tests:
1.  **The "Null Feature" Test:** Train using only the Time or Cycle column. The goal here is to establish a baseline of how much information is leaked simply by knowing "where" we are in the life of the asset. In many datasets (like C-MAPSS), degradation is almost perfectly correlated with the cycle count.
2.  **Permutation Importance:** Identify if a single sensor is a proxy for the target. We need to see which sensors are "carrying" the prediction. If one sensor is a mathematical proxy for the Remaining Useful Life (RUL), the model will exploit it.
3.  **The Chronological Gap Test:** Introduce a "gap" of 10-20 cycles between training and testing. If the score collapses, leakage via temporal proximity is confirmed. This is the "stress test" for temporal leakage. We simulate a real-world scenario where we don't have the "immediate past" data points during the prediction phase

In [ ]:
# --- 1. THE NULL FEATURE TEST ---

# 1. Dataset Setup (Using your confirmed names)
train_data_ag = globals()['train_data_ag']
# If test_data_ag isn't defined, we use a slice of train for the test demonstration
test_data_ag = globals().get('test_data_ag', train_data_ag)

target_col = 'target_rul'

# 2. Construct the Null DataFrames
# We keep only ID, Timestamp, and the RUL target
train_null_df = train_data_ag.reset_index()[['item_id', 'timestamp', target_col]]
test_null_df = test_data_ag.reset_index()[['item_id', 'timestamp', target_col]]

train_null = TimeSeriesDataFrame.from_data_frame(
    train_null_df, id_column='item_id', timestamp_column='timestamp'
)
test_null = TimeSeriesDataFrame.from_data_frame(
    test_null_df, id_column='item_id', timestamp_column='timestamp'
)

# 3. Fit the Null Predictor
# Updated 'eval_metric' to 'MAE' to resolve the ValueError
predictor_null = TimeSeriesPredictor(
    label=target_col,
    eval_metric="MAE",
    prediction_length=1
)

predictor_null.fit(
    train_null,
    hyperparameters={'DirectTabular': {}},
    time_limit=300
)

# 4. Results Comparison
null_leaderboard = predictor_null.leaderboard(test_null)
print("\n--- NULL FEATURE TEST RESULTS ---")
print(null_leaderboard[['model', 'score_test', 'score_val']])

original_score = -2.472684e-09
null_score = null_leaderboard.loc[null_leaderboard['model'] == 'DirectTabular', 'score_test'].values[0]

print(f"\nOriginal Model Score: {original_score}")
print(f"Null Model Score:     {null_score}")

In [ ]:
# --- 2. PERMUTATION IMPORTANCE ---

# 1. Determine the number of unique items (engines/assets) in your test set
num_items = test_data_ag.num_items

# 2. Run Permutation Importance with an explicit integer subsample_size
# We use all items (num_items) to ensure the rigour you requested
importance = predictor.feature_importance(
    data=test_data_ag,
    model='DirectTabular',
    subsample_size=num_items
)

print("\n--- TEST #2: PERMUTATION IMPORTANCE RESULTS ---")
# Displaying the raw importance values
print(importance.sort_values(by='importance', ascending=False))

# 3. Identify if sensors are actually contributing
sensor_importance = importance[importance.index.str.contains('feat')]['importance'].sum()
total_importance = importance['importance'].sum()

print(f"\nTotal Importance: {total_importance}")
print(f"Total Sensor Contribution: {sensor_importance}")

if total_importance > 0:
    sensor_ratio = (sensor_importance / total_importance) * 100
    print(f"Sensor Contribution Ratio: {sensor_ratio:.2f}%")

In [ ]:
# --- 3. THE CHRONOLOGICAL GAP TEST ---

# 1. Setup Data
# We will take the test data and remove the last 20 observations for each item
# to see if the model can still predict the "distant" future.
gap_size = 20
test_data_ag = globals()['test_data_ag']

def create_gapped_test(df, gap):
    # For each item_id, drop the last 'gap' steps
    return df.groupby(level=0, group_keys=False).apply(lambda x: x.iloc[:-gap])

gapped_test_data = create_gapped_test(test_data_ag, gap_size)

print(f"Original Test Steps: {len(test_data_ag)}")
print(f"Gapped Test Steps:   {len(gapped_test_data)} (Gap of {gap_size} steps created)")

# 2. Evaluate the original predictor on the gapped data
# We check if the 'DirectTabular' model can survive without the immediate 'neighbor' data
try:
    gapped_leaderboard = predictor.leaderboard(gapped_test_data)
    print("\n--- TEST #3: CHRONOLOGICAL GAP RESULTS ---")
    print(gapped_leaderboard[gapped_leaderboard['model'] == 'DirectTabular'][['model', 'score_test']])

    gapped_score = gapped_leaderboard.loc[gapped_leaderboard['model'] == 'DirectTabular', 'score_test'].values[0]
    original_score = -2.472684e-09

    print(f"\nOriginal (No Gap) Score: {original_score}")
    print(f"Gapped (20-step) Score:  {gapped_score}")

    if np.isclose(gapped_score, original_score, atol=1e-7):
        print("\nCONCLUSION: The model is a pure Linear Counter. Even with a gap, it just follows the time-slope.")
    else:
        print("\nCONCLUSION: Performance collapsed. The model was relying on immediate temporal proximity (Interpolation).")

except Exception as e:
    print(f"Evaluation failed: {e}")

### AutoGluon Model Appraisal: Critical Methodological Failure

#### 1. Executive Summary: The "Lazy Learner" Pathology
Comprehensive stress testing of the `DirectTabular` model (and the `WeightedEnsemble`) confirms a catastrophic methodological failure. While the leaderboard suggests near-perfect accuracy ($10^{-9}$), this score is a mathematical artifact of **Temporal Interpolation** and **Metadata Leakage**. The model is not performing Health Management; it is performing simple arithmetic on a timestamp.

#### 2. Evidence from Rigorous Stress Testing
| Test | Objective | Result | Finding |
| :--- | :--- | :--- | :--- |
| **1. Null Feature Test** | Remove all 63 sensors; provide only the 'Clock'. | **$-2.47 \times 10^{-9}$** | The model ignores sensors entirely; it maps time to RUL. |
| **2. Permutation Importance** | Measure sensor contribution to prediction. | **0.0%** | Sensor features (`feat_0`–`feat_62`) have zero impact on the score. |
| **3. Chronological Gap** | Force a 20-step gap between context and target. | **Score Collapse** | Accuracy dropped by 7 orders of magnitude (to $-0.0093$). |



#### 3. The Senior Reliability Engineer’s "Litmus Test"
In an aerospace startup environment, presenting this $10^{-9}$ leaderboard to a Senior Reliability Engineer would trigger an immediate red flag. They would pose the following challenge:

> **The Question:** "What happens if a critical sensor, such as **High Pressure Turbine Temperature**, spikes dangerously, but the engine has only been running for 10 hours?"

* **The Model’s Current Answer:** "The engine is fine; RUL is 190 cycles. 10 hours is nowhere near the failure limit."
* **The Physical Reality:** The engine is experiencing a thermal runaway and is seconds from a catastrophic "uncontained" failure.

**Conclusion:** Because the model relies on **Temporal Interpolation** (guessing the next point based on the immediate past) rather than **Degradation Physics**, it is blind to acute anomalies. It predicts safety based on the asset's age, ignoring the physical evidence of imminent breakdown.

#### 4. Required Remediation
To move toward a production-ready Prognostics and Health Management (PHM) system:
1.  **Exclusion of Temporal Bias:** Strictly remove absolute `timestamp` and `cycle` indices from the feature set.
2.  **Differential Feature Engineering:** Transition to training on sensor "deltas" and "gradients" to capture the *physics of failure*.
3.  **Window Shuffling:** Randomise training windows to break the linear continuity that allowed the model to "cheat" via interpolation.

### Transitioning to Condition-Based Monitoring (CBM)

To move from a "Clock Model" to a "Diagnostic Model," we must remove the absolute temporal markers that provided the shortcut for interpolation.

### Why this Change?
By dropping the `timestamp` and `cycle` equivalents, we force AutoGluon to find patterns solely within the **Sensor Manifold** ($feat\_0$ to $feat\_62$). This ensures that if a turbine temperature spikes unexpectedly, the model will recognize the degradation regardless of how many hours the engine has been in service.

### Expected Result
The `score_test` will likely drop (becoming more "negative") because the problem has become significantly harder. However, the resulting **Permutation Importance** will finally show non-zero values for physical sensors, signifying a model that can actually be used for aerospace reliability.

In [ ]:
# --- CONDITION-BASED MONITORING ---

# 1. Prepare Clean Data
# We drop all columns that the Null Test identified as 'clocks'
target_col = 'target_rul'

# Define the sensor features only
sensor_cols = [c for c in train_data_ag.columns if 'feat' in c]

# Create a version of the data that only contains Sensors + Target
# We keep item_id and timestamp ONLY for the index, not as features
train_clean = train_data_ag[sensor_cols + [target_col]]
test_clean = test_data_ag[sensor_cols + [target_col]]

print(f"Retraining with {len(sensor_cols)} sensors and 0 temporal features.")

# 2. Re-initialize the Predictor
# We use a new path to avoid overwriting the 'cheating' model
predictor_physics = TimeSeriesPredictor(
    label=target_col,
    eval_metric="MAE",
    prediction_length=1,
    path="AutogluonModels/ag-physics-only"
)

# 3. Fit with 'DirectTabular'
# This time it MUST find patterns in the sensors
predictor_physics.fit(
    train_clean,
    hyperparameters={'DirectTabular': {}},
    time_limit=600
)

# 4. New Leaderboard
physics_leaderboard = predictor_physics.leaderboard(test_clean)
print("\n--- NEW LEADERBOARD (Physics Only) ---")
print(physics_leaderboard[['model', 'score_test', 'score_val']])

# 5. New Permutation Importance
# This should now show non-zero importance for your sensors
new_importance = predictor_physics.feature_importance(
    data=test_clean,
    model='DirectTabular',
    subsample_size=test_clean.num_items
)
print("\n--- NEW SENSOR IMPORTANCE ---")
print(new_importance.sort_values(by='importance', ascending=False).head(10))

### AutoGluon Model Appraisal: Structural Leakage Confirmed

#### 1. The Paradox of the "Physics-Only" Model
The retraining on physical sensors alone produced a result identical to the previous "Clock" models ($10^{-9}$). However, the **Permutation Importance** remains at absolute zero for every sensor.

#### 2. The Final Diagnosis: Structural Target Leakage
The model is not learning from the columns you provide. Instead, it is exploiting the **index-target relationship**. Because `target_rul` is a perfect linear countdown ($119, 118, 117...$) and the `timestamp` is a perfect linear count up, the `DirectTabular` model's internal preprocessing is creating an identity mapping.

Essentially, the model has discovered that:
$$Target(t) = Target(t-1) - 1$$
As long as the model can see its own previous prediction or the relative position in the time-index, it will produce a "perfect" result that is completely disconnected from the engine sensors.

## ##3. Engineering Reality Check
In an aerospace startup, this model is a **"Phantom Predictor."** * It works perfectly on your historical CSV because the history is a clean countdown.
* It will fail the moment it is put on a real engine where degradation is non-linear, stochastic, or interrupted by maintenance events.

In [ ]:
# 1. Convert TimeSeriesDataFrames to standard DataFrames
# This moves 'item_id' and 'timestamp' into regular columns and removes the temporal ordering
train_shuffled = train_data_ag.reset_index()
test_shuffled = test_data_ag.reset_index()

# 2. Drop the temporal markers to ensure zero leakage
# We remove 'timestamp' and 'item_id' so the model MUST look at sensors
cols_to_drop = ['item_id', 'timestamp']
train_shuffled = train_shuffled.drop(columns=cols_to_drop)
test_shuffled = test_shuffled.drop(columns=cols_to_drop)

# 3. Force Regression Type
# Ensuring target_rul is a float prevents AutoGluon from inferring 'multiclass'
train_shuffled[target_col] = train_shuffled[target_col].astype(float)
test_shuffled[target_col] = test_shuffled[target_col].astype(float)

print(f"Flattened data ready. Training on {len(train_shuffled.columns)-1} sensors.")

In [ ]:
# 1. Fit the Tabular Regression Model
# This will likely take longer than the 10^-9 model because it's actually working
predictor_shuffled = TabularPredictor(
    label=target_col,
    problem_type='regression',
    eval_metric='mae'
).fit(
    train_data=train_shuffled,
    time_limit=300,
    presets='high_quality' # Ensures we get a rigorous look at the sensors
)

# 2. Evaluate and Generate the 'Honest' Importance
shuffled_performance = predictor_shuffled.evaluate(test_shuffled)
shuffled_importance = predictor_shuffled.feature_importance(test_shuffled)

print("\n--- SHUFFLED REGRESSION RESULTS ---")
print(f"Actual MAE (The Honest Score): {shuffled_performance['mean_absolute_error']}")
print("\nTop 10 Physical Sensors:")
print(shuffled_importance.head(10))

### AutoGluon Model Appraisal: The First Genuine Diagnostic

#### 1. Summary of Results
By stripping the temporal index and forcing a **Shuffled Regression**, we have achieved an MAE of **2.28**. Unlike the previous $10^{-9}$ result, this score is derived entirely from the physical state of the asset.

#### 2. Top Diagnostic Drivers (Condition-Based)
| Feature | Importance | Significance (p-value) | Role |
| :--- | :--- | :--- | :--- |
| **feat_41** | **4.378** | $4.45 \times 10^{-7}$ | Primary Health Indicator |
| **feat_23** | **3.476** | $1.29 \times 10^{-7}$ | Secondary Health Indicator |
| **feat_49** | **2.820** | $1.18 \times 10^{-7}$ | Supporting Metric |

#### 3. Aerospace Reliability Implications
We have passed the "Reliability Engineer Litmus Test." If **feat_41** (e.g., Exhaust Gas Temperature or Vibration) spikes, this model will now react because its prediction is mathematically tied to that sensor's value, not the cycle count.

* **Previous Model:** Blind to sensor spikes; followed the clock.
* **Current Model:** Sensitive to sensor deviations; predicts based on asset condition.

### Strategy: From "Snapshot" to "Physics-Aware" Diagnostics

#### 1. The Limitation of Absolute Sensor Values
Absolute sensor readings (e.g., $feat\_41 = 0.85$) are often ambiguous without context. A high temperature might be normal during takeoff but catastrophic during taxiing. Absolute values alone fail to capture the **kinematics of degradation**.


#### 2. Engineering the Physics-Aware Feature Set
We are supplementing our top sensors with temporal context to force the model to focus on **asset behavior** rather than just **asset state**.

| Feature Type | Formula      | Engineering Goal |
| :--- | :--- | :--- |
| **Absolute** | $X_{t}$ | Provides the current operating state |
| **Lag-1** | $X_{t-1}$ | Provides immediate historical context |
| **Delta** | $$\Delta X = X_{t} - X_{t-1}$$    | Captures the **rate of degradation** (Velocity) |
| **Rolling Mean** | $\bar{X} = \frac{1}{k} \sum X_{i}$ | Filters sensor jitter and captures the **trend** |



#### 3. Goal: Preventing "Interpolation Leakage"
By focusing on **Deltas** and **Trends** while strictly excluding absolute timestamps, we ensure the model cannot "cheat" using the clock. The model must now understand that a specific **slope of degradation** in $feat\_41$ correlates to a specific reduction in Remaining Useful Life (RUL).

**The Result:** A robust Condition-Based Monitoring (CBM) model that survives real-world scrutiny from domain experts.

In [ ]:
# 1. Identify the Top Sensors from our Shuffled Regression
top_sensors = ['feat_41', 'feat_23', 'feat_49', 'feat_27', 'feat_53']

def engineer_reliability_features(df):
    # Ensure we are working on a copy to avoid SettingWithCopy warnings
    df = df.copy()

    for sensor in top_sensors:
        # A. Lag-1: The previous state
        df[f'{sensor}_lag1'] = df.groupby(level=0)[sensor].shift(1)

        # B. Delta: The rate of change (Current - Previous)
        # This is the "Velocity" of degradation
        df[f'{sensor}_delta'] = df[sensor] - df[f'{sensor}_lag1']

        # C. Rolling Mean (Window of 5): Smoothing the signal
        # Captures the moving trend and filters out transient sensor noise
        df[f'{sensor}_roll5'] = df.groupby(level=0)[sensor].transform(lambda x: x.rolling(window=5).mean())

    # Fill NaNs created by lagging/rolling with 0 or backfill
    return df.fillna(0)

# 2. Apply engineering to our datasets
train_engineered = engineer_reliability_features(train_data_ag)
test_engineered = engineer_reliability_features(test_data_ag)

# 3. Ensure we drop the target from features (except in the label position)
# And we continue to EXCLUDE the raw timestamp/item_id as features
feature_cols = [c for c in train_engineered.columns if 'feat' in c]
target_col = 'target_rul'

print(f"Feature engineering complete. Total features: {len(feature_cols)}")
print(f"New examples: {train_engineered.filter(like='_delta').columns.tolist()}")

# 4. Final Rigorous Fit
predictor_final = TimeSeriesPredictor(
    label=target_col,
    eval_metric="MAE",
    prediction_length=1,
    path="AutogluonModels/ag-final-physics"
)

predictor_final.fit(
    train_engineered[feature_cols + [target_col]],
    hyperparameters={'DirectTabular': {}},
    time_limit=600
)

# 5. Final Evaluation
final_leaderboard = predictor_final.leaderboard(test_engineered)
print("\n--- FINAL LEADERBOARD (Engineered Physics) ---")
print(final_leaderboard[['model', 'score_test', 'score_val']])

In [ ]:
# 1. Preview the newly engineered features
# We select the primary sensor and its new temporal derivatives
inspection_cols = ['feat_41', 'feat_41_lag1', 'feat_41_delta', 'feat_41_roll5', 'target_rul']

print("--- Engineered Feature Preview (First Asset) ---")
# Looking at the first 7 rows to see the Rolling Mean and Lags populate
print(train_engineered[inspection_cols].head(7))

# 2. Statistical Verification
print("\n--- Delta Statistics ---")
# If the mean delta is negative, the sensor is generally decreasing as RUL decreases
print(train_engineered[['feat_41_delta', 'feat_23_delta']].describe().loc[['mean', 'std', 'min', 'max']])

In [ ]:
# --- FINAL TRAINING EXECUTION: PHYSICS-AWARE TIME-SERIES

# 1. Define the full feature set (Original Sensors + New Engineered Features)
# We ensure we are NOT including 'timestamp' or 'item_id' as features
feature_cols = [c for c in train_engineered.columns if 'feat' in c]
target_col = 'target_rul'

print(f"Final Model: Training on {len(feature_cols)} total features.")

# 2. Re-initialize the Final Predictor
# Using a clean path to store the production-ready model
predictor_final = TimeSeriesPredictor(
    label=target_col,
    eval_metric="MAE",
    prediction_length=1,
    path="AutogluonModels/ag-final-production"
)

# 3. Fit the model
# We use DirectTabular because we've engineered the 'Time' context manually
# through Lags and Deltas, so it no longer needs to 'cheat' with the index.
predictor_final.fit(
    train_engineered[feature_cols + [target_col]],
    hyperparameters={'DirectTabular': {}},
    time_limit=900
)

# 4. Final Leaderboard Evaluation
final_leaderboard = predictor_final.leaderboard(test_engineered)
print("\n--- FINAL PRODUCTION LEADERBOARD ---")
print(final_leaderboard[['model', 'score_test', 'score_val']])

# 5. Final Importance Check
# This will show if our 'Delta' and 'Roll' features are outperforming raw sensors
final_importance = predictor_final.feature_importance(
    data=test_engineered,
    model='DirectTabular',
    subsample_size=test_engineered.num_items
)

print("\n--- TOP PRODUCTION FEATURES ---")
print(final_importance.sort_values(by='importance', ascending=False).head(10))

### AutoGluon Model Appraisal: The Time-Series "Black Box" Leakage

#### 1. Executive Summary: The "Lazy Learner" Pathology
Comprehensive stress testing of the `TimeSeriesPredictor` reveals a catastrophic methodological artifact. Despite leaderboards suggesting near-perfect accuracy ($10^{-9}$), this score is a product of **Internal Structural Leakage**. The AutoGluon TimeSeries engine is so efficient at identifying the linear countdown of the `target_rul` within the `TimeSeriesDataFrame` index that it effectively bypasses all physical sensor inputs.

#### 2. The Senior Reliability Engineer’s "Litmus Test"
In an aerospace startup environment, presenting a $10^{-9}$ leaderboard would trigger immediate scrutiny. A domain expert would pose the following challenge:

> **The Question:** "What happens if a critical sensor, such as **High Pressure Turbine Temperature**, spikes dangerously, but the engine has only been running for 10 hours?"

* **The Time-Series Model’s Answer:** "The engine is fine; RUL is 190 cycles. 10 hours is nowhere near the failure limit."
* **The Physical Reality:** The engine is experiencing thermal runaway and is seconds from catastrophic failure.

**Conclusion:** By "counting down" the RUL based on the internal index, the Time-Series model is blind to acute anomalies. It predicts safety based on age, ignoring physical evidence of imminent breakdown.

#### 3. Evidence of Internal Leakage & Remediation


| Approach | Score (MAE) | Sensor Importance | Validity |
| :--- | :--- | :--- | :--- |
| **Initial Time-Series** | $10^{-9}$ | 0.0 | **FAILED** (Index Leakage) |
| **Physics-Aware TS** | $10^{-9}$ | 0.0 | **FAILED** (Internal Index Leakage) |
| **Shuffled Tabular** | **2.28** | **High (feat_41, feat_23)** | **SUCCESS** (True Physics) |

#### 4. The Verdict: The "Tabular-Only" Path
The `TimeSeriesPredictor` is too "smart" for this dataset format. Even when temporal columns are stripped, the internal engine reconstructs the sequence. To deliver a model that captures **Condition-Based Monitoring (CBM)**, one must abandon the Time-Series object for this specific problem and utilize a **Shuffled Tabular Regression**.

The Shuffled Tabular model is the only architecture "blind" enough to be forced to listen to the sensors, yielding an honest, physics-driven baseline of **2.28 MAE**.

In [ ]:
# --- STEP 1: THE ANOMALY INJECTION SCRIPT ---

# 1. Identify a "Healthy" sample (Early in life, high RUL)
# We'll take a row where target_rul is > 100
healthy_sample = test_shuffled[test_shuffled[target_col] > 100].iloc[[0]].copy()
original_rul = healthy_sample[target_col].values[0]

# 2. Inject the Anomaly
# We spike 'feat_41' (our top sensor) to a value far outside normal range
# or simply to its maximum to simulate a component failure
spike_value = train_shuffled['feat_41'].max() * 2
anomalous_sample = healthy_sample.copy()
anomalous_sample['feat_41'] = spike_value

print(f"Injection Complete.")
print(f"Original feat_41: {healthy_sample['feat_41'].values[0]:.4f}")
print(f"Spiked feat_41:   {anomalous_sample['feat_41'].values[0]:.4f}")
print(f"Ground Truth RUL: {original_rul}")

In [ ]:
# --- STEP 2: THE CONFRONTATION (TIME-SERIES VS TABULAR) ---

# 1. Test the 'Honest' Shuffled Tabular Model
tab_pred = predictor_shuffled.predict(anomalous_sample)

# 2. Test the 'Cheating' Time-Series Model
# (Requires converting back to TimeSeriesDataFrame to trigger its internal logic)
from autogluon.timeseries import TimeSeriesDataFrame
# We create a dummy TS frame for the anomalous sample
anom_ts = TimeSeriesDataFrame.from_data_frame(
    test_data_ag.reset_index().iloc[[0]].copy().assign(feat_41=spike_value),
    id_column='item_id',
    timestamp_column='timestamp'
)
ts_pred = predictor.predict(anom_ts)

print("\n--- ANOMALY STRESS TEST RESULTS ---")
print(f"Original Time-Series Prediction: {ts_pred['mean'].values[0]:.2f} cycles")
print(f"Honest Tabular Prediction:      {tab_pred.values[0]:.2f} cycles")

# Logic Check
if abs(ts_pred['mean'].values[0] - original_rul) < 2:
    print("\nVERDICT: The Time-Series model IGNORED the sensor spike. It is purely following the clock.")

if tab_pred.values[0] < original_rul:
    print(f"VERDICT: The Tabular model REACTED to the physics. It reduced RUL by {original_rul - tab_pred.values[0]:.2f} cycles due to the sensor spike.")

### Safety Validation: Anomaly Stress Test Results

#### 1. Objective
To evaluate model reliability when faced with a "Premature Failure" signal. This test determines if a model is performing **Condition-Based Monitoring (CBM)** or simply **Scheduled Accounting**.

#### 2. The Stress Scenario
* **Asset State:** Early Life (Cycle 0, Ground Truth RUL: 119.0).
* **Anomalous Event:** A catastrophic spike injected into **feat_41** (Top Health Indicator).
* **Sensor Delta:** The reading was forced from a baseline of **-0.2238** to a critical **4.5202**.

#### 3. Comparative Response
| Model | Prediction | Delta from Truth | Safety Verdict |
| :--- | :--- | :--- | :--- |
| **Time-Series ($10^{-9}$)** | **117.13 Cycles** | ~2 Cycles | **CRITICAL FAILURE** |
| **Shuffled Tabular (2.28)** | **101.74 Cycles** | **17.26 Cycles** | **VALIDATED** |

#### 4. Final Conclusion for Startup Portfolio
The Time-Series model is **blind to physics**. Despite a catastrophic sensor failure, it chose to trust the internal "Clock," only subtracting the elapsed time.

The **Shuffled Tabular Model**, despite having a "lower" accuracy on paper, demonstrated genuine **prognostic reasoning**. By slashing the RUL by 17.26 cycles in response to the spike, it proved it is the only model capable of protecting high-value aerospace assets from unpredicted failures.

In [ ]:
# --- HONEST DIAGNOSTIC VISUALISATIONS ---

# 1. Generate predictions for the entire shuffled test set
predictions = predictor_shuffled.predict(test_shuffled)
actuals = test_shuffled[target_col]
residuals = actuals - predictions

# Create a figure with two subplots: Regression and Residuals
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(18, 7))

# --- PLOT 1: PREDICTED VS. ACTUAL ---
sns.scatterplot(x=actuals, y=predictions, alpha=0.3, color='#0047AB', ax=ax1, label='Model Predictions')
line_coords = [actuals.min(), actuals.max()]
ax1.plot(line_coords, line_coords, color='#D22B2B', linestyle='--', linewidth=2, label='Perfect Prognostic')

ax1.set_title('Prognostic Performance: Predicted vs. Actual RUL', fontsize=14, fontweight='bold')
ax1.set_xlabel('Ground Truth RUL (Cycles)', fontsize=12)
ax1.set_ylabel('Predicted RUL (Cycles)', fontsize=12)
ax1.legend()
ax1.grid(True, linestyle=':', alpha=0.6)

# Overlay Metrics
r2 = r2_score(actuals, predictions)
mae = mean_absolute_error(actuals, predictions)
stats_text = f'R²: {r2:.4f}\nMAE: {mae:.2f}'
ax1.text(0.05, 0.90, stats_text, transform=ax1.transAxes, fontsize=12,
         fontweight='bold', bbox=dict(facecolor='white', edgecolor='black', alpha=0.8))

# --- PLOT 2: RESIDUAL PLOT ---
sns.scatterplot(x=predictions, y=residuals, alpha=0.3, color='#4B0082', ax=ax2)
ax2.axhline(0, color='#D22B2B', linestyle='--', linewidth=2)

ax2.set_title('Residual Analysis: Error Distribution', fontsize=14, fontweight='bold')
ax2.set_xlabel('Predicted RUL (Cycles)', fontsize=12)
ax2.set_ylabel('Error (Actual - Predicted)', fontsize=12)
ax2.grid(True, linestyle=':', alpha=0.6)

# Summary Text for Residuals
ax2.text(0.05, 0.05, "Points above 0: Underestimating Wear\nPoints below 0: Overestimating Wear",
         transform=ax2.transAxes, fontsize=10, bbox=dict(facecolor='white', alpha=0.7))

plt.tight_layout()
plt.show()

### Final Model Validation: Safety & Statistical Rigour

#### 1. Safety Validation (Anomaly Stress Test)
**Scenario:** Catastrophic spike in **feat_41** at Cycle 0 (Truth RUL: 119.0).
* **Clock-Based Model:** Predicted **117.13** (Failed to see the fault).
* **Physics-Based Model:** Predicted **101.74** (Successfully reacted to the fault).

**Finding:** The Shuffled Tabular model is sensitive to physical degradation, reducing predicted RUL by **17.26 cycles** purely due to sensor deviation.

#### 2. Statistical Validation (Residual Analysis)
While the Anomaly Test proves **sensitivity**, the Residual Plot proves **consistency**.



#### **Interpretation of Errors:**
* **Zero-Line Center:** The errors are balanced around the zero-axis, confirming our model is an **Unbiased Estimator**. It does not systematically overestimate or underestimate safety.
* **Heteroscedasticity Check:** Observe the "spread" of the residuals as Predicted RUL decreases.
    * *In Aerospace:* We expect wider variance at low RUL (near failure) as component wear becomes non-linear and chaotic.
* **Outlier Identification:** Points far from the zero-line indicate specific assets where the sensor-to-wear mapping deviated—likely due to secondary failure modes not captured by the primary sensors.

#### 3. The "PHM" Audit Conclusion
The combination of **Physics Sensitivity** (Test 1) and **Unbiased Residuals** (Test 2) confirms this model as a viable framework for **Prognostics and Health Management (PHM)**. It rejects the "temporal shortcuts" of black-box time-series models in favour of traceable, condition-based diagnostics.

In [ ]:
# --- OUTLIER IDENTIFICATION: FAILURE MODE AUDIT ---

# 1. Calculate residuals and attach to the original test data
# We use the raw test_shuffled before dropping IDs to track specific engines
results_df = test_shuffled.copy()
results_df['predicted_rul'] = predictions
results_df['residual'] = residuals
results_df['abs_residual'] = results_df['residual'].abs()

# 2. Identify the 'Dangerous' Outliers
# (Where Actual RUL is much LOWER than Predicted - Overestimating safety)
dangerous_outliers = results_df.sort_values(by='residual', ascending=True).head(5)

print("--- CRITICAL OUTLIERS: OVERESTIMATED SAFETY ---")
# If you kept item_id in a previous step, it would show up here
print(dangerous_outliers[['feat_41', 'feat_23', 'target_rul', 'predicted_rul', 'residual']])

# 3. Identify the 'Conservative' Outliers
# (Where Actual RUL is much HIGHER than Predicted - Wasted asset life)
conservative_outliers = results_df.sort_values(by='residual', ascending=False).head(5)

print("\n--- CONSERVATIVE OUTLIERS: PREMATURE REPLACEMENT ---")
print(conservative_outliers[['feat_41', 'feat_23', 'target_rul', 'predicted_rul', 'residual']])

### Final Thesis Appraisal: Error Distribution and Outlier Risks

#### 1. Statistical Validity
The **Residual Plot** confirms that the model is **homoscedastic** across the majority of the flight envelope, meaning its predictive reliability is stable. However, the dispersion increases slightly at lower RUL values, a common phenomenon in aerospace where mechanical wear accelerates non-linearly.

#### 2. Risk Categorization
Through the residual audit, we identify two distinct types of predictive failure:

##### **Category A: Overestimated Safety (Negative Residuals)**
* **The Risk:** The model predicts 50 cycles remaining, but the asset fails in 10.
* **Root Cause:** Likely a "Sudden Death" failure mode (e.g., Foreign Object Damage) that doesn't show the typical slow degradation curve the model learned.
* **Mitigation:** Requires a higher safety margin (Confidence Interval) for lower RUL predictions.

##### **Category B: Overestimated Wear (Positive Residuals)**
* **The Risk:** The model predicts 5 cycles remaining, but the asset could have run for 40 more.
* **Root Cause:** Highly resilient assets or "Golden Engines" that perform better than the population average.
* **Economic Impact:** Leads to premature replacement and wasted operational life.

#### 3. Deployment Conclusion
By moving from a "cheating" $10^{-9}$ Time-Series model to this **Honest Tabular Model**, we have traded illusory perfection for **actionable uncertainty**. We can now quantify the risk of a "Category A" failure, providing a rigour that is required for safety-critical aerospace certification.

In [ ]:
# 1. Set the specific Dataset Location
DATA_PATH = "/content/drive/MyDrive/PI/Datasets"
os.makedirs(DATA_PATH, exist_ok=True)

# 2. Refined Engineering Logic (British English / Physics-Centric)
def engineer_reliability_features_v2(df):
    """
    Constructs kinematic features whilst ensuring zero temporal leakage.
    Uses backfilling to maintain physical consistency at sequence onset.
    """
    df = df.copy()
    top_sensors = ['feat_41', 'feat_23', 'feat_49', 'feat_27', 'feat_53']

    for sensor in top_sensors:
        # Grouping by item_id (level 0) to prevent cross-engine leakage
        engine_group = df.groupby(level=0)[sensor]

        # Lag-1: The immediate preceding state
        df[f'{sensor}_lag1'] = engine_group.shift(1)

        # Delta: Current - Previous (Rate of degradation / Velocity)
        df[f'{sensor}_delta'] = df[sensor] - df[f'{sensor}_lag1']

        # Rolling Mean: Window of 5 for signal-to-noise smoothing
        df[f'{sensor}_roll5'] = engine_group.transform(lambda x: x.rolling(window=5).mean())

    # Backfill NaNs at the start of each engine sequence to maintain consistency
    return df.groupby(level=0, group_keys=False).apply(lambda x: x.bfill()).fillna(0)

# 3. Execute Engineering
train_engineered = engineer_reliability_features_v2(train_data_ag)
test_engineered = engineer_reliability_features_v2(test_data_ag)

# 4. Strip Metadata & Sanitise Index
# We explicitly select ONLY the features and target, then drop the index
# to prevent 'item_id' metadata from leaking into the Parquet file.
feature_cols = [c for c in train_engineered.columns if 'feat' in c]
target_col = 'target_rul'
final_cols = feature_cols + [target_col]

train_sanitised = train_engineered[final_cols].reset_index(drop=True)
test_sanitised = test_engineered[final_cols].reset_index(drop=True)

# 5. Export to Parquet (index=False is the final safety gate)
train_sanitised.to_parquet(os.path.join(DATA_PATH, 'train_physics_v1.parquet'), index=False)
test_sanitised.to_parquet(os.path.join(DATA_PATH, 'test_physics_v1.parquet'), index=False)

print(f"✅ Final Forensic Export Complete.")
print(f"Location: {DATA_PATH}")
print(f"Verification: item_id index dropped to prevent metadata leakage.")

# Final Conclusion: Validation Gate 04

## 1. Audit Findings
We have successfully identified that the initial near-perfect performance ($10^{-9}$ MAE) was an artefact of **Structural Temporal Leakage**. The `TimeSeriesPredictor` was utilising the internal indexing of the `TimeSeriesDataFrame` to 'look up' the RUL countdown rather than learning from sensor physics.

## 2. Remediation Strategy
* **Temporal Blindness:** We transitioned to a **Shuffled Tabular Regression** approach to break the model's reliance on the clock.
* **Kinematic Augmentation:** We engineered **Delta** ($\Delta X$) and **Rolling Mean** ($\bar{X}$) features for the top-performing sensors (`feat_41`, `feat_23`, etc.) to provide the model with 'velocity' and 'trend' context.

## 3. Results & Safety Validation
* **Honest Baseline:** Established an MAE of **~2.28**, which is a realistic and physically grounded score for this dataset.
* **Anomaly Sensitivity:** The model proved its prognostic capability by reacting to a synthetic sensor spike with a **17.26 cycle reduction** in RUL, whereas the 'cheating' model ignored the fault entirely.

## 4. Data Readiness
The physics-aware features have been locked into `train_physics_v1.parquet` and `test_physics_v1.parquet` in the Google Drive directory. These datasets are stripped of all temporal indices, ensuring that the SOTA models in Notebook 05 must compete purely on physical signal processing.

# 🌉 Bridge to Notebook 05: SOTA Benchmarking

The Forensic Audit is complete. We have successfully:
1. Identified structural temporal leakage.
2. Remediated the dataset by engineering kinematic derivatives.
3. Established a physically grounded baseline of **2.28 MAE**.
4. Exported the sanitised **v1_physics** tensors to Google Drive.

**Next Objective:** Benchmark the three SOTA architectures:
* **DeepAR:** To quantify aleatoric uncertainty and risk.
* **TFT:** To audit attention weights against thermodynamic signals.
* **PatchTST:** To extract local semantic patterns from multivariate sequences.